In [ ]:
# ADD LAYER COUNT

# Setup

## imports

In [1]:
import os
import sys
import time
import json
from shutil import copy
from sklearn.utils import shuffle
import datetime as dt

In [2]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.layers.wrappers import TimeDistributed
from keras.layers.convolutional import (Conv2D, MaxPooling3D, Conv3D, MaxPooling2D)
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
from keras.utils import to_categorical
from keras.preprocessing.image import img_to_array

Using TensorFlow backend.


In [3]:
import cv2

In [4]:
from contextlib import redirect_stdout

In [5]:
# setup matplotlib to display plots in the notebook
# %matplotlib inline

# third party imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# setup display options
pd.options.display.max_rows = 200
pd.options.display.max_colwidth = 400
pd.options.display.float_format = '{:,.5g}'.format
np.set_printoptions(precision=5, suppress=False)

# setup seaborn to use matplotlib defaults & styles
sns.set()
sns.set(font_scale=1.2)
sns.set_style("whitegrid", {'axes.grid' : False})

## paths

In [6]:
pwd = os.path.dirname(os.getcwd()) + '/'
pwd

'/mnt/seals/'

In [7]:
path_cache = pwd + 'cache/'
path_models = pwd + 'models/'

## setup logging

In [10]:
import logging

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(threadName)-12.12s] [%(levelname)-5.5s]  %(message)s",
    handlers=[
        logging.FileHandler("{0}/{1}.log".format(pwd, "logs")),
        logging.StreamHandler()
    ])

logger = logging.getLogger()

# helper functions

In [11]:
def plot_pic(imgpath):
    plt.imshow(plt.imread(imgpath))
    plt.show()

# Fit models

In [25]:
def fit_model(model_id, architecture, layer_1_sizefactor, layer_2_sizefactor, layer_3_sizefactor, dropout, sequence_length, pretrained_model_name, pooling):

    ###########################
    ### create folder for model 
    ###########################

    path_model = path_models + str(model_id) + '/'
    if not os.path.exists(path_model):
        os.makedirs(path_model)

    ###########################
    ### create train/test split
    ###########################

    x_train, y_train = get_sequence_data_for_vids(vids_train, sequence_length, pretrained_model_name, pooling)
    x_valid, y_valid = get_sequence_data_for_vids(vids_valid, sequence_length, pretrained_model_name, pooling)
    x_test, y_test = get_sequence_data_for_vids(vids_test, sequence_length, pretrained_model_name, pooling)

    # shuffle test and train batches
    x_train, y_train = shuffle(x_train, y_train)
    x_valid, y_valid = shuffle(x_valid, y_valid)

    # CREATE CLASS BALANCE
#     y_train = pd.DataFrame(y_train)
#     keeps = list(y_train[y_train[0]==1].head(int(len(y_train[y_train[0]==1].index)/2)).index)
#     keeps2 = list(y_train[y_train[0] == 0].index)
#     keeps.extend(keeps2)
#     #
#     y_train = y_train.iloc[keeps]
#     y_train = y_train.values
#     x_train = x_train[keeps,:,:]

    NUM_CLASSES = y_train.shape[1]
    NUM_FEATURES = x_train.shape[2]
    SEQ_LENGTH = x_train.shape[1]
    



    ##############################
    ### keep track of model params
    ##############################

    # create dict with model parameters
    results = {}

    results['id'] = str(model_id)

    NUM_EPOCHS = 50
    BATCH_SIZE = 32
    results['fit_batch_size'] = BATCH_SIZE
    
    results['fit_num_classes'] = NUM_CLASSES
    results['model_num_features'] = NUM_FEATURES
    results['model_sequence_length'] = SEQ_LENGTH
    
    results['pretrained_model_name'] = pretrained_model_name
    results['pretrained_model_pooling'] = pooling

    results['shape_y_train'] = str(y_train.shape)
    results['shape_x_train'] = str(x_train.shape)
    results['shape_x_valid'] = str(x_valid.shape)
    results['shape_x_test'] = str(x_test.shape)

    results['model_architecture'] = architecture
    results['model_layer_1_sizefactor'] = layer_1_sizefactor
    results['model_layer_2_sizefactor'] = layer_2_sizefactor
    results['model_layer_3_sizefactor'] = layer_3_sizefactor
    results['model_dropout'] = dropout

    ################
    ### define model
    ################

    if architecture == "LSTM":
        # https://github.com/sagarvegad/Video-Classification-CNN-and-LSTM-/blob/master/train_CNN_RNN.py
        model = Sequential()

        # layer 1 (LSTM layer)
        model.add(LSTM(NUM_FEATURES//layer_1_sizefactor, return_sequences=False, dropout=dropout, input_shape=(SEQ_LENGTH, NUM_FEATURES)))

        # layer 2 (dense)
        if layer_2_sizefactor > 0:
            model.add(Dropout(dropout))
            model.add(Dense(NUM_FEATURES//layer_2_sizefactor, activation='relu'))

        # layer 3 (dense)
        if layer_2_sizefactor > 0 and layer_3_sizefactor > 0:
            model.add(Dropout(dropout))
            model.add(Dense(NUM_FEATURES//layer_3_sizefactor, activation='relu'))

        # final layer
        model.add(Dropout(dropout))
        model.add(Dense(NUM_CLASSES, activation='softmax'))

        # define optimizer and compile model
        opt = Adam()
        model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    if architecture == 'MLP':
        model = Sequential()
        model.add(Flatten(input_shape=(SEQ_LENGTH, NUM_FEATURES)))
        model.add(Dense(NUM_FEATURES//2, activation='relu'))

        if layer_2_sizefactor > 0:
            model.add(Dense(NUM_FEATURES//layer_2_sizefactor, activation='relu'))
            model.add(Dropout(dropout))

        if layer_2_sizefactor > 0 and layer_3_sizefactor > 0:
            model.add(Dense(NUM_FEATURES//layer_3_sizefactor, activation='relu'))
            model.add(Dropout(dropout))

        # final layer
        model.add(Dropout(dropout))
        model.add(Dense(NUM_CLASSES, activation='softmax'))

        # define optimizer and compile model
        opt = Adam()
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

    # save model summary to file
    with open(path_model + 'model_summary.txt', 'w') as f:
        with redirect_stdout(f):
            model.summary()

    # track number of params in model
    results['param_count'] = model.count_params()

    #############
    ### fit model
    #############

    # setup training callbacks
    stopper_patience = 10
    results['fit_stopper_patience'] = stopper_patience
    callback_stopper = EarlyStopping(monitor='val_acc', patience=stopper_patience, verbose=0)
    callback_csvlogger = CSVLogger(path_model + 'training.log')
    callback_checkpointer = ModelCheckpoint(path_model +  'model.h5', monitor='val_acc', 
                                 save_best_only=True, verbose=0)
    
    # start time training
    start = dt.datetime.now()
    results['fit_train_dt_start'] = start.strftime("%Y-%m-%d %H:%M:%S")

    history = model.fit(x_train, y_train, 
              validation_data=(x_valid,y_valid),
              batch_size=BATCH_SIZE,
              epochs=NUM_EPOCHS,
              callbacks=[callback_stopper, callback_checkpointer, callback_csvlogger],
              shuffle=True,
              verbose=0)
    
    # end time training
    end = dt.datetime.now()    
    results['fit_train_dt_end']  = end.strftime("%Y-%m-%d %H:%M:%S")
    results['fit_train_dt_duration']  = str((end - start).total_seconds()).split(".")[0]

    # get number of epochs actually trained (might have early stopped)
    epochs_trained = 0
    epochs_trained = callback_stopper.stopped_epoch - stopper_patience

    results['fit_stopped_early'] = True
    if epochs_trained == 0 and len(history.history) > stopper_patience:
        results['fit_stopped_early'] = False
        epochs_trained = NUM_EPOCHS - 1 

    results['fit_num_epochs'] = epochs_trained
    results['fit_val_acc'] = history.history['val_acc'][epochs_trained]
    results['fit_train_acc'] = history.history['acc'][epochs_trained]
    results['fit_val_loss'] = history.history['val_loss'][epochs_trained]
    results['fit_train_loss'] = history.history['loss'][epochs_trained]
    

    #######################
    ### predict on test set
    #######################
    
    
    # start time inference
    start = dt.datetime.now()
    results['fit_test_dt_start'] = start.strftime("%Y-%m-%d %H:%M:%S")

    # calculate predictions on test set
    predictions = model.predict(x_test)
    
    # end time training
    end = dt.datetime.now()    
    results['fit_test_dt_end']  = end.strftime("%Y-%m-%d %H:%M:%S")
    results['fit_test_dt_duration']  = str((end - start).total_seconds()).split(".")[0]

    # calculate test error 
    pdf = pd.DataFrame(predictions)
    pdf.columns = ['noseal','seal']

    # get filenames for predictions
    filenames = []
    for vid_name in vids_test:
        filenames_vid = list(dfp[dfp['vid'] == vid_name]['path'])
        filenames.extend(filenames_vid[sequence_length-1:])
    pdf['filename'] = filenames
    truth = pd.DataFrame(y_test)
    truth.columns = ['truth_noseal','truth_seal']
    truth = truth[['truth_seal']]
    pdf['prediction'] = pdf['seal'].apply(lambda x: round(x))
    pdf = pd.concat([pdf, truth], axis=1)
    pdf['error'] = (pdf['prediction'] != pdf['truth_seal']).astype(int)

    test_acc = 1 - pdf['error'].mean()

    pdf.to_csv(path_model + 'test_predictions.csv')

    results['fit_test_acc'] = 1 - pdf['error'].mean()
    logger.info("model {} test acc: {}".format(model_id, test_acc))

    ###################################
    ### save experiment results to file
    ###################################
    # log results
    logger.info(json.dumps(results))
    
    with open(path_model + 'params.json', 'w') as f:
        json.dump(results, f)

In [26]:
# pretrained_model_names = ["inception_resnet_v2", "inception_v3", "mobilenetv2_1.00_224", "resnet50", "vgg16", "xception"]
# poolings = ['avg','max']
# sequence_lengths = [1, 3, 5, 10, 15, 20, 40]

# architectures = ['LSTM', "MLP"]
# layer_1_sizefactors = [1,2,4,8]
# layer_2_sizefactors = [0,1,2,4,8]
# layer_3_sizefactors = [0,1,2,4,8]
# dropouts = [0, 0.1, 0.2,0.3,0.4,0.5]

In [62]:
pretrained_model_names = ["inception_resnet_v2"]
poolings = ['avg']
sequence_lengths = [1, 3]

architectures = ["MLP", "SimpleRNN", "GRU", "LSTM", "Convolution1D", "AtrousConvolution1D"]

layer_1_types = ['dense','sequence']
layer_1_sizefactors = [1,2,4,8]

layer_2_types = ['dense','sequence']
layer_2_sizefactors = [0,2,4,8]

layer_3_types = ['dense','sequence']
layer_3_sizefactors = [0,2,4,8]
dropouts = [0.2, 0.5]

In [28]:
experiment_count_total = 0
for pretrained_model_name in pretrained_model_names:
    for pooling in poolings:
        for sequence_length in sequence_lengths:
            for architecture in architectures:
                for layer_1_sizefactor in layer_1_sizefactors:
                    for layer_2_sizefactor in layer_2_sizefactors:
                        for layer_3_sizefactor in layer_3_sizefactors:
                            for dropout in dropouts:
                                if sequence_length == 1 and architecture == "LSTM":
                                    pass
                                else:
                                    experiment_count_total+=1
experiment_count_total

384

In [29]:
model_id = 1
experiment_count = 1

for pretrained_model_name in pretrained_model_names:
    for pooling in poolings:
        for sequence_length in sequence_lengths:
            for architecture in architectures:
                for layer_1_sizefactor in layer_1_sizefactors:
                    for layer_2_sizefactor in layer_2_sizefactors:
                        for layer_3_sizefactor in layer_3_sizefactors:
                            for dropout in dropouts:
                                
                                # skip LSTM experiement if not a sequence
                                if sequence_length == 1 and architecture == "LSTM":
                                    continue
                                
                                # log experiment
                                param_names = ["model_id", "architecture", "layer_1_sizefactor", "layer_2_sizefactor", "layer_3_sizefactor", "dropout", "sequence_length", "pretrained_model_name", "pooling"]
                                param_values = [str(x) for x in [model_id, architecture, layer_1_sizefactor, layer_2_sizefactor, layer_3_sizefactor, dropout, sequence_length, pretrained_model_name, pooling]]
                                experiment_description = ""
                                for c, p in enumerate(param_names):
                                    experiment_description += p + ': ' + param_values[c] + ', '

                                # only run experiment if results not already computed
                                if not os.path.exists(path_models + str(model_id) + '/params.json'):
                                    # run experiment
                                    logging.info("begin experiment {}/{} - {}".format(experiment_count, experiment_count_total, experiment_description))
                                    fit_model(model_id, architecture, layer_1_sizefactor, layer_2_sizefactor, layer_3_sizefactor, dropout, sequence_length, pretrained_model_name, pooling)
                                
                                experiment_count += 1
                                model_id+=1

# Add models

In [23]:
model_id = 9999

In [63]:
pretrained_model_name = "vgg16"
pooling = 'avg'
sequence_length = 20

layer_1_sizefactor = 2
layer_2_sizefactor = 2
layer_3_sizefactor = 2
dropout = 0.2

In [64]:
architecture = 'SimpleRNN'

In [70]:
###########################
### create folder for model 
###########################

path_model = path_models + str(model_id) + '/'
if os.path.exists(path_model):
    rmtree(path_model)
if not os.path.exists(path_model):
    os.makedirs(path_model)

###########################
### create train/test split
###########################

x_train, y_train = get_sequence_data_for_vids(vids_train, sequence_length, pretrained_model_name, pooling)
x_valid, y_valid = get_sequence_data_for_vids(vids_valid, sequence_length, pretrained_model_name, pooling)
x_test, y_test = get_sequence_data_for_vids(vids_test, sequence_length, pretrained_model_name, pooling)

# shuffle test and train batches
x_train, y_train = shuffle(x_train, y_train)
x_valid, y_valid = shuffle(x_valid, y_valid)

# CREATE CLASS BALANCE
#     y_train = pd.DataFrame(y_train)
#     keeps = list(y_train[y_train[0]==1].head(int(len(y_train[y_train[0]==1].index)/2)).index)
#     keeps2 = list(y_train[y_train[0] == 0].index)
#     keeps.extend(keeps2)
#     #
#     y_train = y_train.iloc[keeps]
#     y_train = y_train.values
#     x_train = x_train[keeps,:,:]

NUM_CLASSES = y_train.shape[1]
NUM_FEATURES = x_train.shape[2]
SEQ_LENGTH = x_train.shape[1]


##############################
### keep track of model params
##############################

# create dict with model parameters
results = {}

results['id'] = str(model_id)

NUM_EPOCHS = 50
BATCH_SIZE = 32
results['fit_batch_size'] = BATCH_SIZE

results['fit_num_classes'] = NUM_CLASSES
results['model_num_features'] = NUM_FEATURES
results['model_sequence_length'] = SEQ_LENGTH

results['pretrained_model_name'] = pretrained_model_name
results['pretrained_model_pooling'] = pooling

results['shape_y_train'] = str(y_train.shape)
results['shape_x_train'] = str(x_train.shape)
results['shape_x_valid'] = str(x_valid.shape)
results['shape_x_test'] = str(x_test.shape)

results['model_architecture'] = architecture
results['model_layer_1_sizefactor'] = layer_1_sizefactor
results['model_layer_2_sizefactor'] = layer_2_sizefactor
results['model_layer_3_sizefactor'] = layer_3_sizefactor
results['model_dropout'] = dropout

In [71]:
# https://github.com/sagarvegad/Video-Classification-CNN-and-LSTM-/blob/master/train_CNN_RNN.py
model = Sequential()

# layer 1 (LSTM layer)
model.add(SimpleRNN(NUM_FEATURES//layer_1_sizefactor, return_sequences=True, dropout=dropout, input_shape=(SEQ_LENGTH, NUM_FEATURES)))

if layer_2_sizefactor > 0:
    model.add(SimpleRNN(NUM_FEATURES//layer_2_sizefactor, return_sequences=True, dropout=dropout))

if layer_3_sizefactor > 0:
    model.add(SimpleRNN(NUM_FEATURES//layer_3_sizefactor, dropout=dropout))
    
# final layer
model.add(Dropout(dropout))
model.add(Dense(NUM_CLASSES, activation='softmax'))

# define optimizer and compile model
opt = Adam()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

## lstm 1

In [74]:
# https://github.com/sagarvegad/Video-Classification-CNN-and-LSTM-/blob/master/train_CNN_RNN.py
model = Sequential()

# layer 1 (LSTM layer)
model.add(LSTM(NUM_FEATURES//layer_1_sizefactor, return_sequences=False, dropout=dropout, input_shape=(SEQ_LENGTH, NUM_FEATURES)))
model.add(Dense(NUM_FEATURES//layer_2_sizefactor, activation='relu'))
    
# final layer
model.add(Dropout(dropout))
model.add(Dense(NUM_CLASSES, activation='softmax'))

# define optimizer and compile model
opt = Adam()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [75]:
#############
### fit model
#############

# setup training callbacks
stopper_patience = 10
results['fit_stopper_patience'] = stopper_patience
callback_stopper = EarlyStopping(monitor='val_acc', patience=stopper_patience, verbose=0)
callback_csvlogger = CSVLogger(path_model + 'training.log')
callback_checkpointer = ModelCheckpoint(path_model +  'model.h5', monitor='val_acc', 
                             save_best_only=True, verbose=1)

history = model.fit(x_train, y_train, 
          validation_data=(x_valid,y_valid),
          batch_size=BATCH_SIZE,
          epochs=NUM_EPOCHS,
          callbacks=[callback_stopper, callback_checkpointer, callback_csvlogger],
          shuffle=True,
          verbose=1)

Train on 9086 samples, validate on 2233 samples
Epoch 1/50
9086/9086 [==============================] - 18s 2ms/step - loss: 0.4642 - acc: 0.7732 - val_loss: 0.9810 - val_acc: 0.3166

Epoch 00001: val_acc improved from -inf to 0.31661, saving model to /mnt/seals/models/9999/model.h5
Epoch 2/50
9086/9086 [==============================] - 16s 2ms/step - loss: 0.3833 - acc: 0.8240 - val_loss: 0.6104 - val_acc: 0.6305

Epoch 00002: val_acc improved from 0.31661 to 0.63054, saving model to /mnt/seals/models/9999/model.h5
Epoch 3/50
9086/9086 [==============================] - 16s 2ms/step - loss: 0.3298 - acc: 0.8578 - val_loss: 0.4486 - val_acc: 0.7913

Epoch 00003: val_acc improved from 0.63054 to 0.79131, saving model to /mnt/seals/models/9999/model.h5
Epoch 4/50
9086/9086 [==============================] - 16s 2ms/step - loss: 0.3059 - acc: 0.8692 - val_loss: 0.3389 - val_acc: 0.8786

Epoch 00004: val_acc improved from 0.79131 to 0.87864, saving model to /mnt/seals/models/9999/model.h5

## lstm 2

In [76]:
# https://github.com/sagarvegad/Video-Classification-CNN-and-LSTM-/blob/master/train_CNN_RNN.py
model = Sequential()

model.add(LSTM(NUM_FEATURES//layer_1_sizefactor, return_sequences=True, dropout=dropout, input_shape=(SEQ_LENGTH, NUM_FEATURES)))
model.add(LSTM(NUM_FEATURES//layer_2_sizefactor, return_sequences=False, dropout=dropout))
    
# final layer
model.add(Dropout(dropout))
model.add(Dense(NUM_CLASSES, activation='softmax'))

# define optimizer and compile model
opt = Adam()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [77]:
#############
### fit model
#############

# setup training callbacks
stopper_patience = 10
results['fit_stopper_patience'] = stopper_patience
callback_stopper = EarlyStopping(monitor='val_acc', patience=stopper_patience, verbose=0)
callback_csvlogger = CSVLogger(path_model + 'training.log')
callback_checkpointer = ModelCheckpoint(path_model +  'model.h5', monitor='val_acc', 
                             save_best_only=True, verbose=1)

history = model.fit(x_train, y_train, 
          validation_data=(x_valid,y_valid),
          batch_size=BATCH_SIZE,
          epochs=NUM_EPOCHS,
          callbacks=[callback_stopper, callback_checkpointer, callback_csvlogger],
          shuffle=True,
          verbose=1)

Train on 9086 samples, validate on 2233 samples
Epoch 1/50
9086/9086 [==============================] - 33s 4ms/step - loss: 0.4656 - acc: 0.7769 - val_loss: 0.2662 - val_acc: 0.9104

Epoch 00001: val_acc improved from -inf to 0.91043, saving model to /mnt/seals/models/9999/model.h5
Epoch 2/50
9086/9086 [==============================] - 31s 3ms/step - loss: 0.3662 - acc: 0.8405 - val_loss: 0.4527 - val_acc: 0.7958

Epoch 00002: val_acc did not improve from 0.91043
Epoch 3/50
9086/9086 [==============================] - 31s 3ms/step - loss: 0.3282 - acc: 0.8613 - val_loss: 0.2630 - val_acc: 0.9207

Epoch 00003: val_acc improved from 0.91043 to 0.92073, saving model to /mnt/seals/models/9999/model.h5
Epoch 4/50
9086/9086 [==============================] - 31s 3ms/step - loss: 0.2842 - acc: 0.8805 - val_loss: 0.3580 - val_acc: 0.8683

Epoch 00004: val_acc did not improve from 0.92073
Epoch 5/50
9086/9086 [==============================] - 31s 3ms/step - loss: 0.2822 - acc: 0.8852 - val_l

## 3 stack

In [46]:
#############
### fit model
#############

# setup training callbacks
stopper_patience = 10
results['fit_stopper_patience'] = stopper_patience
callback_stopper = EarlyStopping(monitor='val_acc', patience=stopper_patience, verbose=0)
callback_csvlogger = CSVLogger(path_model + 'training.log')
callback_checkpointer = ModelCheckpoint(path_model +  'model.h5', monitor='val_acc', 
                             save_best_only=True, verbose=1)

history = model.fit(x_train, y_train, 
          validation_data=(x_valid,y_valid),
          batch_size=BATCH_SIZE,
          epochs=NUM_EPOCHS,
          callbacks=[callback_stopper, callback_checkpointer, callback_csvlogger],
          shuffle=True,
          verbose=1)

Train on 9086 samples, validate on 2233 samples
Epoch 1/50
9086/9086 [==============================] - 19s 2ms/step - loss: 0.5982 - acc: 0.7006 - val_loss: 0.3854 - val_acc: 0.8795

Epoch 00001: val_acc improved from -inf to 0.87953, saving model to /mnt/seals/models/9999/model.h5
Epoch 2/50
9086/9086 [==============================] - 18s 2ms/step - loss: 0.4910 - acc: 0.7761 - val_loss: 0.2980 - val_acc: 0.9033

Epoch 00002: val_acc improved from 0.87953 to 0.90327, saving model to /mnt/seals/models/9999/model.h5
Epoch 3/50
9086/9086 [==============================] - 18s 2ms/step - loss: 0.4234 - acc: 0.8101 - val_loss: 0.3401 - val_acc: 0.8961

Epoch 00003: val_acc did not improve from 0.90327
Epoch 4/50
9086/9086 [==============================] - 18s 2ms/step - loss: 0.4153 - acc: 0.8152 - val_loss: 0.4229 - val_acc: 0.8992

Epoch 00004: val_acc did not improve from 0.90327
Epoch 5/50
9086/9086 [==============================] - 19s 2ms/step - loss: 0.3849 - acc: 0.8338 - val_l

## stacked simple

In [43]:
#############
### fit model
#############

# setup training callbacks
stopper_patience = 10
results['fit_stopper_patience'] = stopper_patience
callback_stopper = EarlyStopping(monitor='val_acc', patience=stopper_patience, verbose=0)
callback_csvlogger = CSVLogger(path_model + 'training.log')
callback_checkpointer = ModelCheckpoint(path_model +  'model.h5', monitor='val_acc', 
                             save_best_only=True, verbose=1)

history = model.fit(x_train, y_train, 
          validation_data=(x_valid,y_valid),
          batch_size=BATCH_SIZE,
          epochs=NUM_EPOCHS,
          callbacks=[callback_stopper, callback_checkpointer, callback_csvlogger],
          shuffle=True,
          verbose=1)

Train on 9086 samples, validate on 2233 samples
Epoch 1/50
9086/9086 [==============================] - 14s 1ms/step - loss: 0.5599 - acc: 0.7124 - val_loss: 0.4046 - val_acc: 0.9069

Epoch 00001: val_acc improved from -inf to 0.90685, saving model to /mnt/seals/models/9999/model.h5
Epoch 2/50
9086/9086 [==============================] - 12s 1ms/step - loss: 0.4468 - acc: 0.7965 - val_loss: 0.3334 - val_acc: 0.9060

Epoch 00002: val_acc did not improve from 0.90685
Epoch 3/50
9086/9086 [==============================] - 12s 1ms/step - loss: 0.4118 - acc: 0.8134 - val_loss: 0.3303 - val_acc: 0.8939

Epoch 00003: val_acc did not improve from 0.90685
Epoch 4/50
9086/9086 [==============================] - 12s 1ms/step - loss: 0.4080 - acc: 0.8172 - val_loss: 0.4440 - val_acc: 0.9046

Epoch 00004: val_acc did not improve from 0.90685
Epoch 5/50
9086/9086 [==============================] - 12s 1ms/step - loss: 0.3890 - acc: 0.8282 - val_loss: 0.2766 - val_acc: 0.9145

Epoch 00005: val_acc i

## fit: simpleRNN single

In [40]:
#############
### fit model
#############

# setup training callbacks
stopper_patience = 10
results['fit_stopper_patience'] = stopper_patience
callback_stopper = EarlyStopping(monitor='val_acc', patience=stopper_patience, verbose=0)
callback_csvlogger = CSVLogger(path_model + 'training.log')
callback_checkpointer = ModelCheckpoint(path_model +  'model.h5', monitor='val_acc', 
                             save_best_only=True, verbose=1)

history = model.fit(x_train, y_train, 
          validation_data=(x_valid,y_valid),
          batch_size=BATCH_SIZE,
          epochs=NUM_EPOCHS,
          callbacks=[callback_stopper, callback_checkpointer, callback_csvlogger],
          shuffle=True,
          verbose=1)

Train on 9086 samples, validate on 2233 samples
Epoch 1/50
9086/9086 [==============================] - 8s 896us/step - loss: 0.5561 - acc: 0.7091 - val_loss: 0.4485 - val_acc: 0.8657

Epoch 00001: val_acc improved from -inf to 0.86565, saving model to /mnt/seals/models/9999/model.h5
Epoch 2/50
9086/9086 [==============================] - 7s 773us/step - loss: 0.4393 - acc: 0.7987 - val_loss: 0.4703 - val_acc: 0.7600

Epoch 00002: val_acc did not improve from 0.86565
Epoch 3/50
9086/9086 [==============================] - 7s 770us/step - loss: 0.4142 - acc: 0.8170 - val_loss: 0.3722 - val_acc: 0.9019

Epoch 00003: val_acc improved from 0.86565 to 0.90193, saving model to /mnt/seals/models/9999/model.h5
Epoch 4/50
9086/9086 [==============================] - 7s 773us/step - loss: 0.3955 - acc: 0.8315 - val_loss: 0.3905 - val_acc: 0.9033

Epoch 00004: val_acc improved from 0.90193 to 0.90327, saving model to /mnt/seals/models/9999/model.h5
Epoch 5/50
9086/9086 [==========================

KeyboardInterrupt: 

## comparison: LSTM

In [38]:
from keras.layers.recurrent import LSTM, SimpleRNN, GRU

In [37]:
#############
### fit model
#############

# setup training callbacks
stopper_patience = 10
results['fit_stopper_patience'] = stopper_patience
callback_stopper = EarlyStopping(monitor='val_acc', patience=stopper_patience, verbose=0)
callback_csvlogger = CSVLogger(path_model + 'training.log')
callback_checkpointer = ModelCheckpoint(path_model +  'model.h5', monitor='val_acc', 
                             save_best_only=True, verbose=1)

history = model.fit(x_train, y_train, 
          validation_data=(x_valid,y_valid),
          batch_size=BATCH_SIZE,
          epochs=NUM_EPOCHS,
          callbacks=[callback_stopper, callback_checkpointer, callback_csvlogger],
          shuffle=True,
          verbose=1)

Train on 9086 samples, validate on 2233 samples
Epoch 1/50
9086/9086 [==============================] - 16s 2ms/step - loss: 0.3640 - acc: 0.8393 - val_loss: 0.3798 - val_acc: 0.8665

Epoch 00001: val_acc improved from -inf to 0.86655, saving model to /mnt/seals/models/9999/model.h5
Epoch 2/50
9086/9086 [==============================] - 16s 2ms/step - loss: 0.3300 - acc: 0.8606 - val_loss: 0.2510 - val_acc: 0.9225

Epoch 00002: val_acc improved from 0.86655 to 0.92253, saving model to /mnt/seals/models/9999/model.h5
Epoch 3/50
9086/9086 [==============================] - 16s 2ms/step - loss: 0.3238 - acc: 0.8661 - val_loss: 0.2940 - val_acc: 0.9109

Epoch 00003: val_acc did not improve from 0.92253
Epoch 4/50
9086/9086 [==============================] - 16s 2ms/step - loss: 0.3127 - acc: 0.8717 - val_loss: 0.2513 - val_acc: 0.9301

Epoch 00004: val_acc improved from 0.92253 to 0.93014, saving model to /mnt/seals/models/9999/model.h5
Epoch 5/50
9086/9086 [==============================

# Examine results

In [36]:
results = []
for folder, subs, files in os.walk(path_models):        
    for filename in files:
        if filename == 'params.json':
            with open(os.path.abspath(os.path.join(folder, filename))) as f:
                data = json.load(f)
            results.append(data)

results = pd.DataFrame(results)

results.sort_values("fit_test_acc", inplace=True, ascending=False)

In [37]:
results.sort_values("fit_val_acc", inplace=True, ascending=False)

In [38]:
results.head(20).T

,284,325,75,345,78,176,22,180,375,54,369,414,243,139,356,278,14,418,302,394
dt_duration_seconds,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155,829,734,NaN,NaN,NaN,794,423,NaN,NaN,278,NaN,126
dt_end,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-16 19:15:04,2018-12-16 22:20:45,2018-12-16 23:24:46,NaN,NaN,NaN,2018-12-16 21:03:18,2018-12-17 09:56:39,NaN,NaN,2018-12-17 09:24:11,NaN,2018-12-16 17:37:25
dt_start,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-16 19:12:29,2018-12-16 22:06:55,2018-12-16 23:12:32,NaN,NaN,NaN,2018-12-16 20:50:04,2018-12-17 09:49:36,NaN,NaN,2018-12-17 09:19:33,NaN,2018-12-16 17:35:18
fit_batch_size,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32
fit_num_classes,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
fit_num_epochs,28,9,11,37,32,5,22,14,35,28,15,5,22,35,36,16,17,20,18,16
fit_stopped_early,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
fit_stopper_patience,10,10,10,10,10,10,10,5,10,10,10,10,10,10,10,10,10,10,10,5
fit_test_acc,0.92,0.94717,0.97333,0.94182,0.89831,0.92982,0.86441,0.82274,0.92308,0.86288,0.94983,0.94314,0.88302,0.87291,0.88294,0.91228,0.89091,0.87625,0.89298,0.92977
fit_test_dt_duration,2,1,2,1,4,0,0,NaN,NaN,NaN,0,4,2,NaN,NaN,0,1,NaN,2,NaN


In [ ]:
results.to_csv(pwd+'results.csv')

# Analyze results of balanced vs unbalanced fits

In [ ]:
# results2 = pd.read_csv(pwd + 'results/results1.csv', index_col=0)

# results['type'] = 'unbalanced'
# results2['type'] = 'balanced'

# results = pd.concat([results,results2],axis=0)

# results.head().T